In [2]:
#importing the libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable  #for stochastic gradient decent 

In [ ]:
#####################---------------------DATA PREPROCESSSING-----------------------###################
#importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep= '::', header= None, engine= 'python', encoding= 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep= '::', header= None, engine= 'python', encoding= 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep= '::', header= None, engine= 'python', encoding= 'latin-1')

In [ ]:
#we use 5 train test splits to do 5 fold cross validation 
#but we will focus on the autoencoders here so we only use 1 train test split
training_set = pd.read_csv('ml-100k/u1.base', delimiter= '\t', engine = 'python', header= None)#delimiter tab arg should be taken with delimiter arg instead of sep
#we need to convert the training_set into an array because pytorch tensors only take arrays as input 
training_set = np.array(training_set, dtype = 'int')

test_set = pd.read_csv('ml-100k/u1.test', delimiter= '\t', engine = 'python', header = None)
test_set = np.array(test_set, dtype = 'int')


In [ ]:
#get max no of user and movies
nb_users = int(max(max(training_set[:, 0]),max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]),max(test_set[:, 1])))

In [ ]:
#making a functoin to convert data into an array with users in rows and movies in columns 
#instead of making a 2d array, we make a list of list 
def convert(data):
    new_data= []#initializing the list 
    for id_users in range(1,nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings)) #we make a list of list because thats what otrch expects
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
#converting list into a torch tensor
#training st and  test set will be 2 differnet tensors
training_set= torch.FloatTensor(training_set)# float tesnors 
test_set = torch.FloatTensor(test_set)
#converted the sets from a numpy array into a torch tensor

In [ ]:
#creating the architecture (class) of the neural network
#a simple functin wouldnt be enough so we make a class 
#we are gonna use inheritance 
#the SAE class is gonna be a child class of a module nn from torch.nn
#we use this so we can use alll variables and dfucntion from the parent clss module because it contains tools to make an AE (optim)
#WE ARE USING STACKED AUTOENCODERS
class SAE(nn.Module):
    def __init__(self, ):  #__init__ fcuntions are like the constructors of the class ie when the obj of the class is made the __init__ function is called#self refers to the obj of the class#in def __init__(self, )we put a commma space nothing to consdider the variables of the module class
        super(SAE, self).__init__() #super() is used to access the inherited methods of the nn.module class
        self.fc1 = nn.Linear(nb_movies, 20)               #first full connection is associated with out obj #first input is the no of input features, , here the features are the movies, in our data set one observation has all the movies
                                                        #the next is the no of nodes/neurons in the first hidden layers, experimentally tuned is 20 (encoding)
        self.fc2 = nn.Linear(20,10)                         #second full connection   (encoding)         
        self.fc3 = nn.Linear(10,20)                      #after the 2nd encoding we have decided to decode in the 3rd one ie reconstruct (decoding)
        self.fc4 = nn.Linear(20,nb_movies)               #4th layer we decode
        self.activation = nn.Sigmoid()#now we specify the activation function ie someone gives good ratings for horror movies it will activate specific feature for the horror genre, this activiation is done byt he actiation fucniton
                                    #from the recifier and the sigmoid actiavation funciton from experimentation in this situation sigmoid performed better hence using sigmoid
                                    # now we make another fucntoin
                                    #U SEE WE NEED 2 FUNCTIONS FOR AEs so we defo needded to make a class
                                    #this other function performs the encoding and decoding as its propogates / moves forward the model
                                    #it will return the vector of predicted ratings that will be compared to the real ratings
    def forward(self, x):           #'x' is our input vector of features with ratings of a specific user 
        x = self.activation(self.fc1(x))  
        x = self.activation(self.fc2(x))  #x here is the vector fo the first hidden layer here we make the 2nd full connection which will encode the vector of 
                                          #20 elements into 10 elements at the same time applying sigmoid acitivation funciton to activate the neurons
        x = self.activation(self.fc3(x))   #same priniciple but now we DECODE into 20 elements again
        x = self.fc4(x)                    #as this isthe final part of the decode we dont use the activation funciton 
                                           #this the predicted vector of ratings which with be compared to the real ratings, measure the loss and update the weights to reduce the loss
        return x
sae = SAE() #creating the sae obj
criterion = nn.MSELoss()#criterion for the loss fucntion, here we use MEAN SQUARED ERROR 
optimizer = optim.RMSprop(sae.parameters(), lr= 0.01, weight_decay= 0.5)                    #we need an optimizer to adjust the weights, an optimizer that applied stochastic grad descent, order to reduce the loss at each epoch
                                #from adam optimizer and RMSprop class we use RMSprop as observed via experimentation
                                #lr is learning rate, from experimentation learnign rate 0.01 is fine, decay is used to reduce learning rate to regulate the convergance
#if we wanna build a autoencoder model we just have to change the architecture here
                                



In [ ]:
###############---------------TRAINIGN THE AUTOENCODER-----------------------##################
nb_epoch = 200 #from experimentation we get 200 as optimal 
for epoch in range(1,nb_epoch+1):
    train_loss = 0 #no loss at the beginning of training
    s=0. #to exclude users who havent rated a single movie #since rms error is a float therefore all means to compute rms error shoul de afloat not compulsary but to avoid error
    for id_user in range (nb_users): #loop that will loop over all the observation #range goes from 0 to nb_users last excluded
        input = Variable(training_set[id_user]).unsqueeze(0)#trainging sets id users is a vector and a network in pytorch or in keras generally cannot accep =y a single vecotr of 1d it accpects a batch of input vectors ie that when we paplythe different functions of a netwrok the function wont take simple vecotrs of 1d as input 
                                      #therefore to an addition dimension we add dimension to it which will correspond to a batch,just like in cnn 
                                      #0 here is the index of the new dimension 
                                      #all thsi will creata  batch of a single input vector 
                                      #the batch can have several input vectors (batch learning)
                                      #but here we do online learning ie batch of 1 input vector but we have to create this batch otherwise it wont work
        target = input.clone()        #same as input vector since we are modifiying hte input vector we would like to have the original input before the modifications
                                      #itll be a clone of input
        if torch.sum(target.data > 0) > 0: #to save memory and exclude users who havent rated a single movie
            output = sae(input)#computation starts now, first step to get a vecgor of predicted ratings ie our output at the very 
                                #right of hte network after the input ie observation went into the network
                                #input is gonna be the 'x' int he forward method which returns 'x' the output 
            target.requires_grad = False   #optimization step 1
                                           #since we have the input and target being hte same thing, to apply stocashtic grad descent we want
                                           #  to make sure the gradient is computed only on the input NOT the target
            output[target == 0] =0         #optim step 2 to only include non zero values ie only for output vector
                                           #as not rated is depicted as '0' in the training set we dont want that to affect the mean score
                                           # we are allowed to dot hsi because these values wont count will not on the computation of th eerror == no impact on the updation of the weights 
            loss = criterion(output, target)#to compute the loss, it takes 2 para the vector of pred ratings (output) and vector of real ratings (target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0)+ 1e-10)#to get the no of movies that have non zero ratings
                                                                               #to make sure the denominator is never zero we use +1e-10, if its zeroo itll create a infinite computatoin that we want to avoid
                                                                               #to avoid bias we add a very very small number that tends to zero but isnt zero
                                                                               #mean_ corrector rep the avg of the error but onyl considering hte rated movies (1-5 ratings)
                                                                               #this mean has to be considered on only the movies that were considered ir no zero rating movies
            loss.backward() #backward methods tells us which direction to update the weights (increase or decrease)
            train_loss += np.sqrt(loss.data*mean_corrector)                #next step is the compute the RMSE and to update the train loss
            s += 1.                                                           #to get no of users who rated atleast 1 movie
            optimizer.step()#diff between backward and optimizer step
            #backward decides the direction of updation of weights
            #optimizer decides the intensity of the updates ie the amounts
            #both the direction and magnitude
    print(f'epoch: {str(epoch)} loss: {str(train_loss/s)}') #trainloss/s to get the average train loss after each epoch
    #if we wanna build a autoencoder model we just have to change the architecture, here we only need to change the no of epochs!
    #loss: the average of the difference between the real ratings and the predictedd ratings ie if loss is '1' that means difference of 1 star:0
    #this loss is for the training set, we should get similar resutls ont he test set unless there is high overfitting ( use k-fold cross validation to avoid)

    


In [ ]:
##########TESTING THE SAE ON THE TEST SET 
test_loss = 0
s=0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      test_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
print('test loss: '+ str(test_loss/s))

In [3]:
torch.rand(2,4)

tensor([[0.3878, 0.9864, 0.4583, 0.1049],
        [0.5264, 0.4472, 0.4492, 0.5288]])